In [ ]:
# | default_exp _cli

In [ ]:
# | export

import importlib
import sys
import asyncio
from pathlib import Path
from typing import *
import signal
from os import getpid
import time
import anyio
import threading
import copy
from contextlib import contextmanager
import multiprocessing
import subprocess

import typer
from fastapi import FastAPI

from fastkafka.application import FastKafka
from fastkafka._components.logger import get_logger, supress_timestamps
from fastkafka._components.helpers import _import_from_string
from fastkafka.server import run_fastkafka_server, terminate_asyncio_process

In [ ]:
import os
from contextlib import contextmanager
from tempfile import TemporaryDirectory

import nbformat
from nbconvert import PythonExporter
from typer.testing import CliRunner

from fastkafka.testing import change_dir
from fastkafka.server import generate_app_in_tmp

In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
runner = CliRunner()

In [ ]:
# | export

_app = typer.Typer(help="")


@_app.command(
    help="Runs Fast Kafka API application",
)
def run(
    num_workers: int = typer.Option(multiprocessing.cpu_count(), help="Number of FastKafka instances to run, defaults to number of CPU cores."),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
) -> None:
    try:
        asyncio.run(run_fastkafka_server(num_workers=num_workers, app=app))
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)


@_app.command(
    help="Creates documentation for a Fast Kafka API application ",
)
def generate_docs(
    root_path: str = typer.Option(
        ".", help="root path under which documentation will be created"
    ),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
) -> None:
    try:
        application = _import_from_string(app)
        application.skip_docs = False
        application.create_docs()
    except Exception as e:

        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

In [ ]:
result = runner.invoke(_app, ["run", "--help"])

                                                                                                                   
 Usage: root run [OPTIONS] APP                                                                                     
                                                                                                                   

Runs Fast Kafka API application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --num-workers        INTEGER  Number of FastKafka instances to run, defaults to number of CPU cores.            │
│                               [default: 4]                                                                      │
│ --help                        Show this message and exit.                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
with generate_app_in_tmp() as app:
    proc = await asyncio.create_subprocess_exec(
                "fastkafka", "run", app, stdout=asyncio.subprocess.PIPE
            )
    time.sleep(5)
    proc.terminate()
    await proc.wait()
    outputs, errs = await proc.communicate()
    
    print(outputs.decode("utf-8"))
    assert proc.returncode == 0, proc.returncode

[684]: [INFO] main: check
[684]: [INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmpuu119_k3/asyncapi/spec/asyncapi.yml' does not exist.
[682]: [INFO] main: check
[682]: [INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmpuu119_k3/asyncapi/spec/asyncapi.yml' does not exist.
[684]: [INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmpuu119_k3/asyncapi/spec/asyncapi.yml'
[684]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[684]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[684]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[684]: [INFO] fastkafka.application: _create_producer() : created producer using the config: '{

In [ ]:
result = runner.invoke(_app, ["generate-docs", "--help"])

                                                                                                                   
 Usage: root generate-docs [OPTIONS] APP                                                                           
                                                                                                                   

Creates documentation for a Fast Kafka API application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --root-path        TEXT  root path under which documentation will be created [default: .]                       │
│ --help                   Show this message and exit.                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
with generate_app_in_tmp() as import_str:
        result = runner.invoke(_app, ["generate-docs", import_str])
        typer.echo(result.output)
        assert result.exit_code == 0

        result = runner.invoke(_app, ["generate-docs", import_str])
        typer.echo(result.output)
        assert result.exit_code == 0

[INFO] main: check
[INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmpmca42uek/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmpmca42uek/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka._components.asyncapi: Async docs generated at 'asyncapi/docs'
[INFO] fastkafka._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'npm WARN deprecated har-validator@5.1.5: this library is no longer supported
npm WARN deprecated uuid@3.4.0: Please upgrade  to version 7 or higher.  Older versions may use Math.random() in certain circumstances, which is known to be problematic.  See https://v8.dev/blog/math-random for details.
npm WARN deprecated readdir-scoped-modules@1.1.0: This functionality has been moved to @npmcli/fs
npm WARN deprecated request@2.88.2: request has been deprecated, see https://github.com